[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/autocaption-colab/blob/main/autocaption_colab.ipynb)

In [ ]:
!git clone -b dev https://github.com/camenduru/autocaption

!apt update -y
!apt install -qq imagemagick

!pip install -q faster_whisper==0.7.0 ffmpeg==1.4 ffmpeg_python==0.2.0 googletrans==3.1.0a0 openai_whisper==20230314 pydub==0.25.1 
!pip install -q spacytextblob==4.0.0 SpeechRecognition==3.10.0 streamlit==1.25.0 streamlit_extras==0.3.0 moviepy==2.0.0.dev2

!ldconfig /usr/lib64-nvidia

%cd /content/autocaption

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

!sed -i '/@\\*/d' $(identify -list policy | grep 'policy.xml' | awk '{print $NF}') # thanks to @chigozienri ❤

os.environ["IMAGEMAGICK_BINARY"] = "/usr/bin/convert"
id_rsa_file = "/content/id_rsa"
id_rsa_pub_file = "/content/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

gen_key(id_rsa_file)

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:8501 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

!streamlit run app.py